In [23]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import shap
import sys, os
import time
from tqdm.notebook import tqdm
import pandas as pd

from tensorflow.keras.layers import (Input, Layer, Dense)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K

sys.path.insert(0, '/scratch/nj594/xai/helpers')
from evaluate import evaluate

import pickle

# IMPORTANT: SET RANDOM SEEDS FOR REPRODUCIBILITY
os.environ['PYTHONHASHSEED'] = str(420)
import random
random.seed(420)
np.random.seed(420)
tf.random.set_seed(420)

In [19]:
#Set Model Dir
method = 'gradcam'
model_dir = 'gradcam'
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

## Load ECGs, Predictions, and Labels

In [3]:
data_dir = os.path.join(os.getcwd(), 'data')

X_val = np.load(os.path.join(data_dir, 'X_val_processed.npy'), allow_pickle=True)
X_test = np.load(os.path.join(data_dir, 'X_test_processed.npy'), allow_pickle=True)

y_val = np.load(os.path.join(data_dir, 'y_val.npy'), allow_pickle=True)
y_test = np.load(os.path.join(data_dir, 'y_test.npy'), allow_pickle=True)

preds = np.load(os.path.join(data_dir, 'predictions.npy'), allow_pickle=True)
preds_discrete = np.eye(5)[preds.argmax(1)]

preds_val = np.load(os.path.join(data_dir, 'predictions_val.npy'), allow_pickle=True)
preds_discrete_val = np.eye(5)[preds_val.argmax(1)]

INPUT_SHAPE = (544, 544, 3)
NUM_CLASSES = 5

In [26]:
BATCH_SIZE = 32
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, Resizing
import tensorflow_datasets as tfds

(ds_train, ds_val, ds_test), ds_info = tfds.load(
    'diabetic_retinopathy_detection/btgraham-300',
    split=['train', 'validation', 'test'],
    as_supervised=False,
    with_info=True,
)

def batch_data(dataset, fn, batch_size=32):
    dataset = dataset.map(fn)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

def reformat(input_dict):
    image = tf.cast(input_dict['image'], tf.float32)
    image = Resizing(INPUT_SHAPE[0], INPUT_SHAPE[1], interpolation='nearest')(image)
    image = tf.keras.applications.inception_v3.preprocess_input(image)
    label = tf.one_hot(input_dict['label'], depth = NUM_CLASSES)

    return (image, label)

ds_train = batch_data(ds_train, reformat, BATCH_SIZE)
ds_val = batch_data(ds_val, reformat, BATCH_SIZE)
ds_test = batch_data(ds_test, reformat, BATCH_SIZE)

# Load Model

In [27]:
model.evaluate(ds_test)

2022-04-17 20:20:21.494533: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 20:20:21.498254: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 20:20:21.502646: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 20:20:21.516199: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 20:20:21.519908: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 20:20:21.524221: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 20:20:22.112415: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 20:20:22.169347: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


1333/1334 [============================>.] - ETA: 0s - loss: 0.6402 - auroc: 0.9474 - auprc: 0.8383 - accuracy: 0.8060

MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record

MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record

MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record

MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record

MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record

1334/1334 [==============================] - 441s 330ms/step - loss: 0.6401 - auroc: 0.9474 - auprc: 0.8383 - accuracy: 0.8060


MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486


[0.64011150598526, 0.9474074244499207, 0.8382940292358398, 0.8059995174407959]

In [28]:
evaluator_model.evaluate(ds_test)

2022-04-17 20:27:42.527102: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 20:27:42.531084: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 20:27:42.535622: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 20:27:42.549974: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 20:27:42.553775: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 20:27:42.558262: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 20:27:43.162147: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 20:27:43.220461: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


1334/1334 [==============================] - 329s 246ms/step - loss: 0.5385 - auroc: 0.9601 - auprc: 0.8722 - accuracy: 0.8221


[0.5385178327560425,
 0.9600709676742554,
 0.8722018599510193,
 0.8221232891082764]

In [ ]:
model.evaluate(X_test, y_test)

In [21]:
evaluator_model.evaluate(X_test, y_test)

2022-04-17 20:16:56.735340: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 20:16:56.739308: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 20:16:56.742224: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 20:16:58.075083: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


79/79 [==============================] - 21s 242ms/step - loss: 3.8483 - auroc: 0.5036 - auprc: 0.2022 - accuracy: 0.2032


[3.8482558727264404,
 0.5036420822143555,
 0.2021995186805725,
 0.20319999754428864]

In [4]:
# Load Model
from tensorflow.keras.applications.densenet import DenseNet121

base_model = DenseNet121(
    include_top=False, weights='imagenet', 
    input_shape=INPUT_SHAPE, pooling='avg'
)

base_model.trainable = True
base_model.summary()

model_input = Input(shape=INPUT_SHAPE, name='input')

net = base_model(model_input)
out = Dense(NUM_CLASSES, activation='softmax')(net)

model = Model(model_input, out)

model.load_weights('model/densenet/model_weights.h5')
model.trainable = False

OPTIMIZER = tf.keras.optimizers.Adam(1e-3)
METRICS = [ 
  tf.keras.metrics.AUC(name='auroc'),
  tf.keras.metrics.AUC(curve='PR', name='auprc'),
  tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='accuracy'),
]

model.compile(
    loss='categorical_crossentropy',
    optimizer=OPTIMIZER,
    metrics=METRICS,
)

2022-04-17 17:37:23.410095: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-17 17:37:23.410662: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 32252 MB memory:  -> device: 0, name: Vega 20, pci bus id: 0000:8d:00.0


Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 544, 544, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 550, 550, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 272, 272, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 272, 272, 64) 256         conv1/conv[0][0]                 
________________________________________________________________________________________

# GradCAM

### Explain

In [5]:
from tf_explain.core.grad_cam import GradCAM
import cv2

submodel_index, submodel = 1, model.layers[1]
x = submodel.outputs[0]
for layer_index in range(submodel_index+1, len(model.layers)):
    extracted_layer = model.layers[layer_index]
    print('Surgically appending layer : '+str(extracted_layer))
    x = extracted_layer(x)
flat_model = Model(inputs=submodel.inputs, outputs=x)

layer_name = 'conv5_block16_concat'
cams = []
t = time.time()
grad_model = tf.keras.models.Model(
    flat_model.inputs, [flat_model.get_layer(layer_name).get_output_at(0), flat_model.output]
)

explanations = []
for y_class in range(y_test.shape[1]):
    cams = []
    for i in tqdm(range(100)): #mini-batch
        with tf.GradientTape() as tape:
            inputs = tf.cast(X_test[i*25:(i+1)*25], tf.float32)
            tape.watch(inputs)
            conv_outputs, predictions = grad_model(inputs)
            loss = predictions[:, y_class]

        grads = tape.gradient(loss, conv_outputs)
        grads = (
            tf.cast(conv_outputs > 0, "float32")
            * tf.cast(grads > 0, "float32")
            * grads
        )

        cam = tf.stack(GradCAM.generate_ponderated_output(conv_outputs, grads))
        cams.append(cam)

    cams = np.array([cv2.resize(cam, (544, 544)) for cam in tf.concat(cams, 0).numpy()])
    explanations.append(cams)

explaining_time = time.time() - t

with open(os.path.join(method, 'explanations.pkl'), 'wb') as f:
    pickle.dump(explanations, f)

with open(os.path.join(method, 'explaining_time.pkl'), 'wb') as f:
    pickle.dump(explaining_time, f) 

Surgically appending layer : <keras.layers.core.Dense object at 0x7efee45c6510>


# Evaluation

In [8]:
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# Load Evaluator Model

eval_dir = os.path.join(os.getcwd(), 'evaluation', 'evaluator-data')
evaluator_model = tf.keras.models.load_model(os.path.join(eval_dir, 'surrogate.h5'))

OPTIMIZER = tf.keras.optimizers.Adam(1e-3)
METRICS = [ 
  tf.keras.metrics.AUC(name='auroc'),
  tf.keras.metrics.AUC(curve='PR', name='auprc'),
  tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='accuracy'),
]

evaluator_model.compile(
    loss='categorical_crossentropy',
    optimizer=OPTIMIZER,
    metrics=METRICS,
)

In [9]:
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### Retrospective Evaluation ####

# Exclusion
retro_ex_test = evaluate(X_test, explanations, evaluator_model, y_test, y_test, 
                         mode = 'exclude', method = method)

# Inclusion
retro_in_test = evaluate(X_test, explanations, evaluator_model, y_test, y_test, 
                         mode = 'include', method = method)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### Prospective Evaluation ####

# Exclusion
pro_ex_test = evaluate(X_test, explanations, evaluator_model, preds_discrete, y_test, 
                         mode = 'exclude', method = method)

# Inclusion
pro_in_test = evaluate(X_test, explanations, evaluator_model, preds_discrete, y_test, 
                         mode = 'include', method = method)

100


2022-04-17 18:08:54.416655: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-04-17 18:08:54.436925: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 18:08:54.441123: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 18:08:54.443720: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 18:08:55.650135: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIO

99


2022-04-17 18:09:45.733847: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 18:09:45.738192: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 18:09:45.740755: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


95


...

2022-04-17 18:48:06.346530: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 18:48:06.350465: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 18:48:06.352803: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


1


2022-04-17 18:48:56.010533: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 18:48:56.014590: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 18:48:56.016913: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


0


2022-04-17 18:49:23.126878: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 18:49:23.130918: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-04-17 18:49:23.133215: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [12]:
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Combine Results
tags = ['retro_ex','retro_in','pro_ex','pro_in']
result_list = [retro_ex_test,retro_in_test,pro_ex_test,pro_in_test]

results = {}
for res, tag  in zip(result_list, tags):
    res = {k+'-'+tag:v for k,v in res.items()}
    results = {**results, **res}

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# Save

### Create Results Dictionary
header = ["model_dir", "explaining_time"]
metrics = ['AUC_acc','AUC_auroc','AUC_log_likelihood','AUC_log_odds']
for tag in tags:
    header += [x+'-'+tag for x in metrics]
    
results['model_dir'] = model_dir
results["explaining_time"] = explaining_time
results = {k:v for k,v in results.items() if k in header}

### Convert to DataFrame
results_df = pd.DataFrame(results, index=[0])
results_df = results_df[header]

In [16]:
retro_in_test

{'acc': {100: 0.2032,
  99: 0.2244,
  95: 0.236,
  90: 0.2492,
  85: 0.2596,
  75: 0.2676,
  50: 0.3024,
  25: 0.3268,
  15: 0.3328,
  10: 0.3396,
  5: 0.3184,
  1: 0.228,
  0: 0.2},
 'auroc': {100: 0.5057004,
  99: 0.5287872,
  95: 0.5550234,
  90: 0.5730758,
  85: 0.5881562,
  75: 0.6144246000000001,
  50: 0.6724028000000001,
  25: 0.7302955,
  15: 0.748447,
  10: 0.7481421,
  5: 0.7313052000000001,
  1: 0.6652518,
  0: 0.4998},
 'preds': {100: array([[8.93752098e-01, 8.83543417e-02, 1.78448595e-02, 4.39195537e-05,
          4.84293469e-06],
         [9.08368826e-01, 4.09528874e-02, 5.03622368e-02, 2.39651723e-04,
          7.64532961e-05],
         [1.27890427e-02, 1.67057589e-02, 6.70378923e-01, 1.46436870e-01,
          1.53689399e-01],
         ...,
         [1.77840471e-01, 4.10715304e-02, 7.09612131e-01, 5.46135008e-02,
          1.68623831e-02],
         [9.32725132e-01, 4.54182364e-02, 2.16827337e-02, 1.02160004e-04,
          7.17712683e-05],
         [8.97488832e-01, 7.4732

In [14]:
### Append DataFrame to csv
results_path = method+'/results.csv'
if os.path.exists(results_path):
    results_df.to_csv(results_path, mode='a',  header=False)
else:
    results_df.to_csv(results_path, mode='w',  header=True)

In [17]:
results_df

,model_dir,explaining_time,AUC_acc-retro_ex,AUC_auroc-retro_ex,AUC_log_likelihood-retro_ex,AUC_log_odds-retro_ex,AUC_acc-retro_in,AUC_auroc-retro_in,AUC_log_likelihood-retro_in,AUC_log_odds-retro_in,AUC_acc-pro_ex,AUC_auroc-pro_ex,AUC_log_likelihood-pro_ex,AUC_log_odds-pro_ex,AUC_acc-pro_in,AUC_auroc-pro_in,AUC_log_likelihood-pro_in,AUC_log_odds-pro_in
0,gradcam,1866.558584,0.11926,0.337236,-4.342654,-3.955996,0.291764,0.662496,-2.529064,-1.948323,0.200878,0.504611,-3.106368,-2.700708,0.200208,0.499958,-3.696025,-3.153029
